In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

snscolors = sns.color_palette("colorblind")

In [2]:
from cosmicfishpie.fishermatrix import cosmicfish

In [6]:
fiducial = {
    "Omegam": 0.32,
    "Omegab": 0.05,
    "h": 0.67,
    "ns": 0.96,
    "sigma8": 0.815584,
    "w0": -1.0,
    "wa": 0.0,
    "mnu": 0.06,
    "Neff": 3.044,
}

options = {
    "accuracy": 1,
    "feedback": 1,
    "code": "camb",
    "specs_dir": "../survey_specifications/",
    "outroot": "3x2pt_presentation",
    "specs_dir": "../survey_specifications/",
    "survey_name": "Euclid",
    "survey_name_photo": "Euclid-Photometric-ISTF-Pessimistic",
    "cosmo_model": "w0waCDM",
}

In [7]:
observables = ["GCph", "WL"]

### Pass options and settings to Cosmicfishpie

We set the survey to `Euclid-Photometric-ISTF-Pessimistic` setting the default settings for the galaxy distributions, biases and modeling choices

In [ ]:
cosmoFM_A = cosmicfish.FisherMatrix(
    fiducialpars=fiducial,
    options=options,
    observables=observables,
    cosmoModel=options["cosmo_model"],
    surveyName=options["survey_name"],
)

# Photometric 3x2pt Angular Power Spectrum Observable

The 3x2pt function is the auto- and cross-correlation of observables X, Y = {L, G},  which can be combined in 3 times the 2pt-function of:
Lensing-Lensing, Galaxy-Lensing and Galaxy-Galaxy

$$
C_{ij}^{XY}(\ell) = \int^{z_{max}}_{z_{min}} \!\!\!\!\! dz  \frac{W_{i}^{X}(z) W_{j}^{Y}(z)}{H(z)r^{2}(z)} P_{\delta \delta} \! \left[ \frac{\ell+1/2}{r(z)},z \right] +N_{ij}^{XY}(\ell)
$$

where the cosmic shear window function is given by:

$$
W_i^{\gamma}(z) = \frac{3}{2} \, H_0^2 \, \Omega_m \,  (1+z) \, r(z) \int_z^{z_{max}}dz^\prime \,
    %\niof(z') 
    n_i(z^\prime)
    \left[1- \frac{r(z)}{r(z^\prime)}\right]~
$$

and the estimated number density of galaxies in each bin is given by:

$$
n_i(z) = \frac{\int_{z_i^-}^{z_i^+} dz_p n(z) p_{ph}(z_p|z)}{\int_{z_{min}}^{z_{max}}dz \int_{z_i^-}^{z_i^+}dz_p n(z) p_{ph}(z_p|z)}~
$$

which is a convolution of the theoretical mean number density with a photometric redshift error distribution:

$$
p_{ph}(z_p|z) = \frac{1-f_{out}}{\sqrt{2\pi}\sigma_b(1+z)} \exp\left\{-\frac{1}{2}\left[\frac{z-c_bz_p-z_b}{\sigma_b(1+z)}\right]^2\right\} \\ + \frac{f_{out}}{\sqrt{2\pi}\sigma_0(1+z)} \exp\left\{-\frac{1}{2}\left[\frac{z-c_0z_p-z_0}{\sigma_0(1+z)}\right]^2\right\}
$$

The full lensing function is given by the addition of cosmic shear and intrinsic alignment, which takes the following form:

$$
W_{i}^{L}(z) = W_{i}^{\gamma}(z) - W_{i}^\mathrm{IA}(z) 
$$

Moreover for IA and for photometric Galaxy clustering, the window function takes the simple form:

$$
W_{i}^\mathrm{IA}(z) = \mathcal{A}_\mathrm{IA} \mathcal{C}_\mathrm{IA}\Omega_{m,0} \frac{\mathcal{F}_\mathrm{IA}(z)}{D(z)} n_i(z) H(z)\\
W_{i}^{G}(z) = n_i(z) H(z)\, b_i(z)
$$

Here, $b_i(z)$ is the galaxy bias (the ratio between the density contrast of galaxies and dark matter).

In [ ]:
print(cosmoFM_A.biaspars)
print(cosmoFM_A.photopars)
print(cosmoFM_A.IApars)

# Compute the Photometric angular power spectrum $C(\ell)$

In [12]:
from cosmicfishpie.LSSsurvey import photo_obs as phobs

Declare an instance of the class and compute the angular power spectrum

In [13]:
photo_Cls = phobs.ComputeCls(
    cosmoFM_A.fiducialcosmopars, cosmoFM_A.photopars, cosmoFM_A.IApars, cosmoFM_A.biaspars
)
phCls_A = photo_Cls.compute_all()

### Extract the radial selection functions from Cosmicfishpie

In [17]:
zz = np.linspace(0.001, 2.5, 250)  ## Euclid specs are only defined up to z=3.

In [18]:
ph_window = photo_Cls.window

In [ ]:
# Full window function and binned photo n(z) distribution
window = photo_Cls.window

color = iter(snscolors)
plt.figure(figsize=(12, 9))
plt.plot(zz, window.dNdz(zz), label=r"$n(z)$", color="k", lw=3, ls="--")
for ind in photo_Cls.binrange:
    c = next(color)
    plt.plot(zz, window.ngal_photoz(zz, ind), label=r"$n^{ph}_{%s}$" % ind, color=c)
    plt.plot(zz, window.n_i(zz, ind), ls=":")  # label=r'dNi bin-'+str(ind), color=c)
plt.xlabel(r"$z$", fontsize=25)
plt.ylabel(r"$n(z)$", fontsize=25)
plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", fontsize=20)

In [ ]:
# Testing the generation of kernels and Cls

color = iter(snscolors)
fig, axs = plt.subplots(3, 1, figsize=(13, 9))
plt.figure(figsize=(10, 8))
for ind in photo_Cls.binrange:
    c = next(color)
    axs[0].plot(zz, photo_Cls.genwindow(zz, "GCph", ind)[0], lw=2.0, color=c)
    axs[1].plot(zz, photo_Cls.genwindow(zz, "WL", ind)[0], lw=3.0, color=c)
    axs[2].plot(zz, photo_Cls.genwindow(zz, "WL", ind)[1], lw=3.0, color=c, label=r"bin %s" % ind)
axs[0].set_xlabel(r"$z$", fontsize=20)

axs[0].set_ylabel(r"$W_i^{G}(z)$", fontsize=20)
axs[1].set_ylabel(r"$W_i^{\gamma}(z)$", fontsize=20)
axs[2].set_ylabel(r"$W_i^\mathrm{IA}(z)$", fontsize=20)
axs[0].minorticks_on()
axs[1].minorticks_on()
axs[2].minorticks_on()
fig.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", fontsize=20)

## Finally plot the 3x2pt $C^{XY}_{ij}(\ell)$

In [ ]:
# Accessing LSS cls
cls = photo_Cls.result
ell = cls["ells"]
norm = ell * (ell + 1) / (2 * np.pi)

color = iter(snscolors)
fig, axs = plt.subplots(3, 1, figsize=(21, 25), sharex=True)
for bin1, bin2 in zip([1, 2, 4, 6, 10], [2, 2, 5, 7, 10]):
    c = next(color)
    axs[0].loglog(
        ell,
        norm * cls["GCph " + str(bin1) + "xGCph " + str(bin2)],
        color=c,
        label="GxG i=" + str(bin1) + ", j=" + str(bin2),
        ls="-",
        lw=2.1,
    )
    axs[1].loglog(
        ell,
        norm * cls["WL " + str(bin1) + "xWL " + str(bin2)],
        color=c,
        label="LxL i=" + str(bin1) + ", j=" + str(bin2),
        ls="-",
        lw=2.1,
    )
    corr = norm * cls["WL " + str(bin1) + "xGCph " + str(bin2)]
    if np.mean(corr) < 0:
        ls = "--"
    else:
        ls = "-"
    axs[2].loglog(
        ell, abs(corr), color=c, label="LxG i=" + str(bin1) + ", j=" + str(bin2), ls=ls, lw=2.1
    )
axs[0].set_ylabel(r"$\ell(\ell+1)C_{ij}^{GG}(\ell)/2\pi$", fontsize=35)
axs[0].legend(bbox_to_anchor=(1.04, 0.5), loc="center left", fontsize=25)
axs[1].set_ylabel(r"$\ell(\ell+1)C_{ij}^{LL}(\ell)/2\pi$", fontsize=35)
axs[1].legend(bbox_to_anchor=(1.04, 0.5), loc="center left", fontsize=25)
axs[2].set_ylabel(r"$\ell(\ell+1)C_{ij}^{LG}(\ell)/2\pi$", fontsize=35)
axs[2].legend(bbox_to_anchor=(1.04, 0.5), loc="center left", fontsize=25)

axs[2].set_xlim(10, 740)
axs[2].set_xlabel(r"Multipole $\ell$", fontsize=35)
fig.subplots_adjust(hspace=0.0)

[ax.tick_params(which="major", length=15, width=2, direction="in") for ax in axs]
[ax.tick_params(which="minor", length=8, width=1, direction="in") for ax in axs]
[ax.minorticks_on() for ax in axs]
[ax.tick_params(axis="both", which="major", labelsize=21) for ax in axs]
[ax.tick_params(axis="both", which="minor", labelsize=15) for ax in axs]

plt.show()